In [83]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, DirGNNConv
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
import random, h5py, pickle, glob, os
import networkx as nx
from copy import deepcopy

output_dir = r"C:\Users\uhewm\Desktop\ProjectHGT\simulation_chunks(4)"
all_files = sorted(glob.glob(os.path.join(output_dir, "*.h5")))

graphs = []

for file in random.sample(all_files, 10000):
    with h5py.File(file, "r") as f:
        grp = f["results"]
        graph_properties = pickle.loads(grp["graph_properties"][()])
        
        nodes = torch.tensor(graph_properties[0])              # [num_nodes]
        edges = torch.tensor(graph_properties[1], dtype=torch.long)  # [2, num_edges]
        coords = torch.tensor(graph_properties[2].T)           # [2, num_nodes]
        #edges_reversed = edges.flip(0)  # Tauscht Zeile 0 und 1

        # Erstelle einen gerichteten Graphen
        G = nx.DiGraph()
        
        # Füge Knoten hinzu (optional mit Koordinaten als Attribut)
        for i, node_id in enumerate(nodes.tolist()):
            G.add_node(node_id, core_distance = coords[:, i].tolist()[0], allele_distance = coords[:, i].tolist()[1])
        
        # Füge Kanten hinzu
        edge_list = edges.tolist()
        for src, dst in zip(edge_list[0], edge_list[1]):
            G.add_edge(src, dst)

        H = deepcopy(G)
            
        for node in G.nodes():
            children = list(G.predecessors(node))
            if children:
                core_sum = sum(G.nodes[child]['core_distance'] for child in children)
                allele_sum = sum(G.nodes[child]['allele_distance'] for child in children)
                H.nodes[node]['core_distance'] = G.nodes[node]['core_distance'] - core_sum
                H.nodes[node]['allele_distance'] = G.nodes[node]['allele_distance'] - allele_sum
            else:
                H.nodes[node]['core_distance'] = G.nodes[node]['core_distance']
                H.nodes[node]['allele_distance'] = G.nodes[node]['allele_distance']


        node_features = []
        for node in list(H.nodes):
            core = H.nodes[node].get("core_distance", 0.0)
            allele = H.nodes[node].get("allele_distance", 0.0)
            node_features.append([core, allele])
        coords = torch.tensor(node_features, dtype=torch.float32).T
         
        # Node features: nur die zwei Werte pro Knoten (coords)
        x_node_features = coords.float().T  # Shape [num_nodes, 2]

        # Labels
        theta_gains = torch.tensor(
            [1 if node in grp.attrs["parental_nodes_hgt_events_corrected"] else 0 
             for node in graph_properties[0]],
            dtype=torch.long
        )

        # PyG-Graph erstellen
        data = Data(
            x=x_node_features,       # Node Features [num_nodes, 2]
            edge_index=edges,        # Edge Index [2, num_edges]
            y=theta_gains            # Labels [num_nodes]
        )
        graphs.append(data)


C:\Users\uhewm\AppData\Local\Temp\ipykernel_21540\2309232405.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edges = torch.tensor(graph_properties[1], dtype=torch.long)  # [2, num_edges]
C:\Users\uhewm\AppData\Local\Temp\ipykernel_21540\2309232405.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coords = torch.tensor(graph_properties[2].T)           # [2, num_nodes]


In [81]:
#### FUNKTIONIERT!

# Train/Test Split
random.shuffle(graphs)
split_idx = int(0.8 * len(graphs))
train_graphs = graphs[:split_idx]
test_graphs = graphs[split_idx:]

train_loader = DataLoader(train_graphs, batch_size=8, shuffle=True)
test_loader = DataLoader(test_graphs, batch_size=8)

# === 2. Modell definieren ===
class GCNClassifier(nn.Module):
    def __init__(self, in_channels, hidden_channels, dropout=0.3):
        super().__init__()
        # Innerer conv wird an DirGNNConv übergeben
        self.conv1 = DirGNNConv(GCNConv(in_channels, hidden_channels))
        self.conv2 = DirGNNConv(GCNConv(hidden_channels, hidden_channels))
        self.lin = nn.Linear(hidden_channels, 1)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        #x = self.conv2(x, edge_index)
        #x = F.relu(x)
        x = self.lin(x)
        return x.view(-1)


# === 3. Modell, Optimizer, Loss ===
model = GCNClassifier(in_channels=2, hidden_channels=32)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Klassengewichte berechnen (gegen Ungleichgewicht)
all_labels = torch.cat([g.y for g in train_graphs])
ratio = (len(all_labels) - all_labels.sum()) / all_labels.sum()
pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

print(f"Pos Weight: {pos_weight.item():.2f}")

# === 4. Training & Evaluation ===
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

@torch.no_grad()
def evaluate(loader):
    model.eval()
    total_correct = 0
    total_nodes = 0
    tp, fp, fn = 0, 0, 0

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        preds = torch.sigmoid(out) > 0.5
        total_correct += (preds == batch.y.bool()).sum().item()
        total_nodes += batch.y.size(0)

        # Metriken für Klasse 1
        tp += ((preds == 1) & (batch.y == 1)).sum().item()
        fp += ((preds == 1) & (batch.y == 0)).sum().item()
        fn += ((preds == 0) & (batch.y == 1)).sum().item()

    acc = total_correct / total_nodes
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return acc, precision, recall, f1

# === 5. Training starten ===
for epoch in range(1, 51):
    loss = train()
    acc, prec, rec, f1 = evaluate(test_loader)
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

C:\Users\uhewm\AppData\Local\Temp\ipykernel_21540\3794257270.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)


Pos Weight: 6.65
Epoch 01 | Loss: 2.5878 | Acc: 0.921 | Prec: 0.197 | Rec: 0.929 | F1: 0.325
Epoch 02 | Loss: 0.8332 | Acc: 0.940 | Prec: 0.246 | Rec: 0.942 | F1: 0.390
Epoch 03 | Loss: 0.6011 | Acc: 0.957 | Prec: 0.313 | Rec: 0.931 | F1: 0.468
Epoch 04 | Loss: 0.4696 | Acc: 0.967 | Prec: 0.377 | Rec: 0.919 | F1: 0.534
Epoch 05 | Loss: 0.3687 | Acc: 0.974 | Prec: 0.440 | Rec: 0.920 | F1: 0.595
Epoch 06 | Loss: 0.3162 | Acc: 0.978 | Prec: 0.481 | Rec: 0.931 | F1: 0.634
Epoch 07 | Loss: 0.2549 | Acc: 0.983 | Prec: 0.555 | Rec: 0.913 | F1: 0.690
Epoch 08 | Loss: 0.2091 | Acc: 0.985 | Prec: 0.578 | Rec: 0.904 | F1: 0.705
Epoch 09 | Loss: 0.1818 | Acc: 0.986 | Prec: 0.601 | Rec: 0.898 | F1: 0.720
Epoch 10 | Loss: 0.1575 | Acc: 0.985 | Prec: 0.595 | Rec: 0.898 | F1: 0.716
Epoch 11 | Loss: 0.1459 | Acc: 0.987 | Prec: 0.623 | Rec: 0.885 | F1: 0.731
Epoch 12 | Loss: 0.1324 | Acc: 0.986 | Prec: 0.611 | Rec: 0.891 | F1: 0.725
Epoch 13 | Loss: 0.1234 | Acc: 0.987 | Prec: 0.636 | Rec: 0.855 | F1: 0

In [37]:
# Train/Test Split
random.shuffle(graphs)
split_idx = int(0.8 * len(graphs))
train_graphs = graphs[:split_idx]
test_graphs = graphs[split_idx:]

train_loader = DataLoader(train_graphs, batch_size=8, shuffle=True)
test_loader = DataLoader(test_graphs, batch_size=8)

# === 2. Modell definieren ===
class GCNClassifier(nn.Module):
    def __init__(self, in_channels, hidden_channels, dropout=0.3):
        super().__init__()
        self.conv1 = DirGNNConv(GCNConv(in_channels, hidden_channels))
        self.conv2 = DirGNNConv(GCNConv(hidden_channels, hidden_channels))
        self.conv3 = DirGNNConv(GCNConv(hidden_channels, hidden_channels))
        self.conv4 = DirGNNConv(GCNConv(hidden_channels, hidden_channels))
        self.lin = nn.Linear(hidden_channels, 1)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv4(x, edge_index)
        x = F.relu(x)

        x = self.lin(x)
        return x.view(-1)

# === 3. Modell, Optimizer, Loss ===
model = GCNClassifier(in_channels=2, hidden_channels=32)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Klassengewichte berechnen (gegen Ungleichgewicht)
all_labels = torch.cat([g.y for g in train_graphs])
ratio = (len(all_labels) - all_labels.sum()) / all_labels.sum()
pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

print(f"Pos Weight: {pos_weight.item():.2f}")

# === 4. Training & Evaluation ===
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

@torch.no_grad()
def evaluate(loader):
    model.eval()
    total_correct = 0
    total_nodes = 0
    tp, fp, fn = 0, 0, 0

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        preds = torch.sigmoid(out) > 0.5
        total_correct += (preds == batch.y.bool()).sum().item()
        total_nodes += batch.y.size(0)

        # Metriken für Klasse 1
        tp += ((preds == 1) & (batch.y == 1)).sum().item()
        fp += ((preds == 1) & (batch.y == 0)).sum().item()
        fn += ((preds == 0) & (batch.y == 1)).sum().item()

    acc = total_correct / total_nodes
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return acc, precision, recall, f1

# === 5. Training starten ===
for epoch in range(1, 51):
    loss = train()
    acc, prec, rec, f1 = evaluate(test_loader)
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

Pos Weight: 6.94


C:\Users\uhewm\AppData\Local\Temp\ipykernel_21540\1273750112.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)


Epoch 01 | Loss: 0.6432 | Acc: 0.938 | Prec: 0.246 | Rec: 0.884 | F1: 0.385
Epoch 02 | Loss: 0.2845 | Acc: 0.958 | Prec: 0.328 | Rec: 0.870 | F1: 0.476
Epoch 03 | Loss: 0.1933 | Acc: 0.960 | Prec: 0.342 | Rec: 0.873 | F1: 0.492
Epoch 04 | Loss: 0.1717 | Acc: 0.967 | Prec: 0.389 | Rec: 0.858 | F1: 0.535
Epoch 05 | Loss: 0.1613 | Acc: 0.970 | Prec: 0.411 | Rec: 0.835 | F1: 0.551
Epoch 06 | Loss: 0.1503 | Acc: 0.973 | Prec: 0.437 | Rec: 0.846 | F1: 0.576
Epoch 07 | Loss: 0.1467 | Acc: 0.971 | Prec: 0.428 | Rec: 0.888 | F1: 0.577
Epoch 08 | Loss: 0.1397 | Acc: 0.970 | Prec: 0.419 | Rec: 0.903 | F1: 0.572
Epoch 09 | Loss: 0.1383 | Acc: 0.972 | Prec: 0.434 | Rec: 0.895 | F1: 0.584
Epoch 10 | Loss: 0.1366 | Acc: 0.972 | Prec: 0.437 | Rec: 0.899 | F1: 0.589
Epoch 11 | Loss: 0.1314 | Acc: 0.972 | Prec: 0.430 | Rec: 0.913 | F1: 0.585
Epoch 12 | Loss: 0.1301 | Acc: 0.972 | Prec: 0.433 | Rec: 0.916 | F1: 0.588
Epoch 13 | Loss: 0.1286 | Acc: 0.972 | Prec: 0.435 | Rec: 0.915 | F1: 0.590
Epoch 14 | L

In [86]:
import numpy as np

# === 2. Modell definieren ===
class GCNClassifier(nn.Module):
    def __init__(self, in_channels, hidden_channels, dropout=0.3):
        super().__init__()
        self.conv1 = DirGNNConv(GCNConv(in_channels, hidden_channels))
        self.conv2 = DirGNNConv(GCNConv(hidden_channels, hidden_channels))
        self.conv3 = DirGNNConv(GCNConv(hidden_channels, hidden_channels))
        self.conv4 = DirGNNConv(GCNConv(hidden_channels, hidden_channels))
        self.lin = nn.Linear(hidden_channels, 1)
        self.dropout = dropout

    def forward(self, x, edge_index):
        edge_index = edge_index[[1, 0], :]
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.conv4(x, edge_index)
        x = F.relu(x)

        x = self.lin(x)
        return x.view(-1)


# === 3. Modell, Optimizer, Loss ===
model = GCNClassifier(in_channels=2, hidden_channels=32)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Klassengewichte berechnen (gegen Ungleichgewicht)
all_labels = torch.cat([g.y for g in train_graphs])
ratio = (len(all_labels) - all_labels.sum()) / all_labels.sum()
pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

print(f"Pos Weight: {pos_weight.item():.2f}")

# === 4. Training & Evaluation ===
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

@torch.no_grad()
def evaluate(loader, threshold=0.5):
    model.eval()
    total_correct = 0
    total_nodes = 0
    tp, fp, fn = 0, 0, 0

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        preds = torch.sigmoid(out) > threshold
        total_correct += (preds == batch.y.bool()).sum().item()
        total_nodes += batch.y.size(0)

        tp += ((preds == 1) & (batch.y == 1)).sum().item()
        fp += ((preds == 1) & (batch.y == 0)).sum().item()
        fn += ((preds == 0) & (batch.y == 1)).sum().item()

    acc = total_correct / total_nodes
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return acc, precision, recall, f1

@torch.no_grad()
def find_best_threshold(loader, thresholds=np.linspace(0, 1, 101)):
    model.eval()
    best_threshold = 0.5
    best_f1 = 0.0

    # Alle Outputs und Labels sammeln, damit man nicht für jeden Threshold neu durch die Daten geht
    all_outs = []
    all_labels = []
    for batch in loader:
        out = model(batch.x, batch.edge_index)
        all_outs.append(torch.sigmoid(out))
        all_labels.append(batch.y)
    all_outs = torch.cat(all_outs)
    all_labels = torch.cat(all_labels)

    for threshold in thresholds:
        preds = all_outs > threshold
        tp = ((preds == 1) & (all_labels == 1)).sum().item()
        fp = ((preds == 1) & (all_labels == 0)).sum().item()
        fn = ((preds == 0) & (all_labels == 1)).sum().item()

        precision = tp / (tp + fp + 1e-8)
        recall = tp / (tp + fn + 1e-8)
        f1 = 2 * precision * recall / (precision + recall + 1e-8)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1

@torch.no_grad()
def show_some_predictions(loader, n_samples=3, threshold=0.5):
    model.eval()
    all_probs = []
    all_preds = []
    all_labels = []

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        probs = torch.sigmoid(out)
        preds = (probs > threshold).long()
        all_probs.append(probs)
        all_preds.append(preds)
        all_labels.append(batch.y)

    all_probs = torch.cat(all_probs)
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    total_samples = len(all_labels)
    indices = random.sample(range(total_samples), k=min(n_samples, total_samples))

    for i, idx in enumerate(indices):
        print(f"Sample {i + 1}:")
        print(f"  True label:      {all_labels[idx].item()}")
        print(f"  Predicted prob:  {all_probs[idx].item():.4f}")
        print(f"  Predicted label: {all_preds[idx].item()}")

# === 5. Training starten ===
for epoch in range(1, 51):
    loss = train()
    acc, prec, rec, f1 = evaluate(test_loader, threshold=0.5)
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

# Nach Training besten Threshold bestimmen
best_threshold, best_f1 = find_best_threshold(test_loader)
print(f"\nBester Threshold: {best_threshold:.3f} mit F1-Score: {best_f1:.3f}")

# Evaluation mit bestem Threshold
acc, prec, rec, f1 = evaluate(test_loader, threshold=best_threshold)
print(f"Evaluation mit bestem Threshold:")
print(f"Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

print("\nEin paar Beispielvorhersagen auf Trainingsdaten mit bestem Threshold:")
show_some_predictions(train_loader, n_samples=3, threshold=best_threshold)



C:\Users\uhewm\AppData\Local\Temp\ipykernel_21540\1012065659.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)


Pos Weight: 6.65
Epoch 01 | Loss: 0.6265 | Acc: 0.977 | Prec: 0.468 | Rec: 0.845 | F1: 0.603
Epoch 02 | Loss: 0.2420 | Acc: 0.966 | Prec: 0.367 | Rec: 0.936 | F1: 0.527
Epoch 03 | Loss: 0.1602 | Acc: 0.982 | Prec: 0.530 | Rec: 0.867 | F1: 0.658
Epoch 04 | Loss: 0.1433 | Acc: 0.978 | Prec: 0.483 | Rec: 0.920 | F1: 0.633
Epoch 05 | Loss: 0.1350 | Acc: 0.982 | Prec: 0.532 | Rec: 0.889 | F1: 0.666
Epoch 06 | Loss: 0.1287 | Acc: 0.983 | Prec: 0.552 | Rec: 0.907 | F1: 0.686
Epoch 07 | Loss: 0.1222 | Acc: 0.983 | Prec: 0.548 | Rec: 0.910 | F1: 0.684
Epoch 08 | Loss: 0.1185 | Acc: 0.984 | Prec: 0.560 | Rec: 0.909 | F1: 0.693
Epoch 09 | Loss: 0.1180 | Acc: 0.984 | Prec: 0.567 | Rec: 0.912 | F1: 0.699
Epoch 10 | Loss: 0.1102 | Acc: 0.986 | Prec: 0.605 | Rec: 0.889 | F1: 0.720
Epoch 11 | Loss: 0.1104 | Acc: 0.986 | Prec: 0.601 | Rec: 0.909 | F1: 0.723
Epoch 12 | Loss: 0.1072 | Acc: 0.986 | Prec: 0.601 | Rec: 0.899 | F1: 0.720
Epoch 13 | Loss: 0.1078 | Acc: 0.985 | Prec: 0.595 | Rec: 0.902 | F1: 0

In [75]:
@torch.no_grad()
def show_some_graph_predictions(loader, n_samples=3, threshold=0.5):
    model.eval()
    graphs = []

    # Alle Graphen und deren Vorhersagen sammeln
    for batch in loader:
        out = model(batch.x, batch.edge_index)
        probs = torch.sigmoid(out)
        preds = (probs > threshold).long()
        graphs.append({
            "probs": probs,
            "preds": preds,
            "labels": batch.y
        })

    total_graphs = len(graphs)
    indices = random.sample(range(total_graphs), k=min(n_samples, total_graphs))

    for i, idx in enumerate(indices):
        graph = graphs[idx]
        print(f"Graph Sample {i + 1} (Index {idx}):")
        print("Node-wise True Labels:     ", graph["labels"].tolist())
        #print("Node-wise Predicted Probs: ", [f"{p:.4f}" for p in graph["probs"].tolist()])
        print("Node-wise Predicted Labels:", graph["preds"].tolist())
        print("-" * 50)


print("\nEin paar Beispielvorhersagen auf Trainingsdaten mit bestem Threshold:")
show_some_graph_predictions(train_loader, n_samples=2, threshold=best_threshold)



Ein paar Beispielvorhersagen auf Trainingsdaten mit bestem Threshold:
Graph Sample 1 (Index 18):
Node-wise True Labels:      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [66]:
110148/199

553.5075376884422